# Capstone Assignment 20.1: Initial Report and Exploratory Data Analysis (EDA)

Nathan Oyama

## 1 &emsp; Planning the project

There are three data sets to accomplish this project. 

* Kaggle: "Percent Sunshine by US City". kaggle.com/datasets/thedevastator/annual-percent-of-possible-sunshine-by-us-city.

* US Geological Survey: "The United States Large-Scale Solar Photovoltaic Database (USPVDB)". US Department of the Interior. energy.usgs.gov/uspvdb/data.

* landvalue: "ZHVI 3-Bedroom Time Series($) - City". landvalue.com/research/data.

Then take the following steps _for every dataset_:

1. From the data set which is in CSV format, create a pandas DataFrame object.
1. Analyze every DataFrame and identify which columns to use for this project.
1. Format the DataFrames before merge them.

Finally, merge the three DataFrames into one.

## 2 &emsp; Analyzing Data Sets

Analyze those three data sets.

In [180]:
import pandas as pd
import re

pd.options.mode.copy_on_write = True

### 2.1 Analyzing Data Set 1: "Percent Sunshine by US City"

In [181]:
df_sunshine_original = pd.read_csv(
    './data/Average Percent of Possible Sunshine by US City.csv'
    )

print(df_sunshine_original.shape)
print(df_sunshine_original.head())

(317, 16)
   index           CITY JAN FEB MAR APR MAY JUN JUL AUG SEP OCT NOV DEC ANN  \
0      0  BIRMINGHAM,AL  46  53  57  65  65  67  59  62  59  66  55  49  58   
1      1  MONTGOMERY,AL  47  55  58  64  63  64  61  61  59  63  55  49  58   
2      2   ANCHORAGE,AK  43  46  51  50  51  46  43  43  41  36  35  33  43   
3      3      JUNEAU,AK  39  35  38  42  44  37  33  35  27  21  26  21  33   
4      4        NOME,AK  38  56  54  52  52  43  39  34  38  35  30  36  42   

   Unnamed: 14  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  


You can use all columns except "index" and "Unnamed: 14". The "CITY" column includes the name of the city in all uppercase, followed by a comma (",") and the state abbreviation.

In [182]:
del df_sunshine_original

### 2.2 &emsp; Analyzing Data Set 2: "The US Large-Scale Solar Photovoltaic Database (USPVDB)"

In [183]:
df_photovoltanic_original = pd.read_csv(
    './data/uspvdb_v3_0_20250430.csv'
    )

print(df_photovoltanic_original.shape)

(5712, 26)


In [184]:
print(df_photovoltanic_original.iloc[:,:13].head())

   case_id multi_poly  eia_id p_state           p_county       ylat  \
0   406374     single   66887      AK  Matanuska-Susitna  61.587349   
1   405016      multi    6304      AK   Northwest Arctic  66.838470   
2   401476      multi   60058      AL         Lauderdale  34.833809   
3   401865      multi   60679      AL               Dale  31.331732   
4   401866      multi   60680      AL            Calhoun  33.626301   

        xlong   p_area  p_img_date  p_dig_conf                   p_name  \
0 -149.789413   172005    20240814           4            Houston Solar   
1 -162.553146     8740    20240719           4          Kotzebue Hybrid   
2  -87.838394  1735134    20220212           4    River Bend Solar, LLC   
3  -85.729469   187820    20220609           4  Fort Rucker Solar Array   
4  -85.940590    39717    20210814           4         ANAD Solar Array   

   p_year p_pwr_reg  
0    2023        AK  
1    2020       NaN  
2    2016       TVA  
3    2017      SOCO  
4    2017   

In [185]:
print(df_photovoltanic_original.iloc[:,13:26].head())

  p_tech_pri p_tech_sec p_sys_type       p_axis  p_azimuth  p_tilt  p_battery  \
0         PV        NaN     ground   fixed-tilt      180.0    40.0        NaN   
1         PV        NaN     ground  single-axis      156.0    40.0  batteries   
2         PV       c-si     ground  single-axis      270.0    17.0        NaN   
3         PV  thin-film     ground  single-axis      188.0    20.0        NaN   
4         PV  thin-film     ground   fixed-tilt      180.0    20.0        NaN   

   p_cap_ac  p_cap_dc      p_type       p_agrivolt p_comm  p_zscore  
0       6.0       8.4  greenfield             crop    NaN -0.457675  
1       1.7       3.4  greenfield  non-agrivoltaic    NaN  5.617232  
2      75.0     100.2  greenfield  non-agrivoltaic    NaN -0.298527  
3      10.6      12.7  greenfield  non-agrivoltaic    NaN -0.122265  
4       7.4       9.7   superfund  non-agrivoltaic    NaN  3.031619  


In [186]:
del df_photovoltanic_original

### 2.3 &emsp; Analyzing Data Set 3: "ZHVI 3-Bedroom Time Series($) - City"

In [187]:
df_landvalue_original = pd.read_csv(
    './data/City_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv.zip',
    compression='zip'
    )

df_landvalue_original.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,2002-10-31,2002-11-30,2002-12-31,2003-01-31,2003-02-28,2003-03-31,2003-04-30,2003-05-31,2003-06-30,2003-07-31,2003-08-31,2003-09-30,2003-10-31,2003-11-30,2003-12-31,2004-01-31,2004-02-29,2004-03-31,2004-04-30,2004-05-31,2004-06-30,2004-07-31,2004-08-31,2004-09-30,2004-10-31,2004-11-30,2004-12-31,2005-01-31,2005-02-28,2005-03-31,2005-04-30,2005-05-31,2005-06-30,2005-07-31,2005-08-31,2005-09-30,2005-10-31,2005-11-30,2005-12-31,2006-01-31,2006-02-28,2006-03-31,2006-04-30,2006-05-31,2006-06-30,2006-07-31,2006-08-31,2006-09-30,2006-10-31,2006-11-30,2006-12-31,2007-01-31,2007-02-28,2007-03-31,2007-04-30,2007-05-31,2007-06-30,2007-07-31,2007-08-31,2007-09-30,2007-10-31,2007-11-30,2007-12-31,2008-01-31,2008-02-29,2008-03-31,2008-04-30,2008-05-31,2008-06-30,2008-07-31,2008-08-31,2008-09-30,2008-10-31,2008-11-30,2008-12-31,2009-01-31,2009-02-28,2009-03-31,2009-04-30,2009-05-31,2009-06-30,2009-07-31,2009-08-31,2009-09-30,2009-10-31,2009-11-30,2009-12-31,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,2010-10-31,2010-11-30,2010-12-31,2011-01-31,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,2011-12-31,2012-01-31,2012-02-29,2012-03-31,2012-04-30,2012-05-31,2012-06-30,2012-07-31,2012-08-31,2012-09-30,2012-10-31,2012-11-30,2012-12-31,2013-01-31,2013-02-28,2013-03-31,2013-04-30,2013-05-31,2013-06-30,2013-07-31,2013-08-31,2013-09-30,2013-10-31,2013-11-30,2013-12-31,2014-01-31,2014-02-28,2014-03-31,2014-04-30,2014-05-31,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-10-31,2014-11-30,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30,2025-05-31
0,6181,0,New York,city,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,260254.899991,261634.936152,263035.285487,265755.628388,268537.262690,271540.803319,274684.685195,277577.034679,280225.595389,283007.236196,286304.212798,289526.956545,292654.837159,295515.354541,298638.996994,301808.051312,304718.323281,307522.118851,310207.310883,313169.182122,315895.295344,318551.360488,321587.071324,325155.741965,329092.979419,332385.088893,333194.788341,332854.557386,332395.370390,334067.937165,336510.274755,339775.608296,343119.158839,346568.801946,349275.69508

In [188]:
df_landvalue_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15711 entries, 0 to 15710
Columns: 313 entries, RegionID to 2025-05-31
dtypes: float64(305), int64(2), object(6)
memory usage: 37.5+ MB


In [189]:
df_landvalue_original.columns

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'Metro', 'CountyName', '2000-01-31', '2000-02-29',
       ...
       '2024-08-31', '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31',
       '2025-01-31', '2025-02-28', '2025-03-31', '2025-04-30', '2025-05-31'],
      dtype='object', length=313)

In [190]:
del df_landvalue_original

### 2.4 &emsp; Planning the merged Data Frame


| Column              | Example          | Data Sets                           |
| :------------------ | :--------------- | :---------------------------------- |
| State               | AL               | Sunshine, Photovoltanic, Land Value |
| County              | DALE             | Photovoltanic, Land Value           |
| City                | BIRMINGAM        | Sunshine, Land Value                |
| Longitude           | -149.789413.     | Photovoltanic                       |
| Latitude            | 61.587349        | Sunshine, Photovoltanic, Land Value |
| Sunshine (Jan ... Dec)  | 58           | Sunshine                            |
| DC                  | 6.0              | Photovoltanic                       |
| AC                  | 8.4              | Photovoltanic                       |
| Current             | 14.4             | (DC + AC)                           |
| Land Value          | 840048.900963529 | Land Value                          |

Some cities that are listed in the Land Value data set and the Sunshine data set do not have solar power plants. In later steps, you predict the current in cities that do not have solar power plants and tell which cities you should build solar power plants.

## 3 &emsp; Extracting Data Sets

### 3.1 &emsp; Dataset 1: Sunshine

In [191]:
df_sunshine_original = pd.read_csv(
    './data/Average Percent of Possible Sunshine by US City.csv'
    )

Check the `CITY` column:

In [192]:
print(df_sunshine_original[['CITY']].value_counts(ascending=True))

CITY                  
CITY                      1
ABERDEEN,SD               2
NOME,AK                   2
NORFOLK,VA                2
NORTH PLATTE,NE           2
                         ..
GRAND RAPIDS,MI           2
GREAT FALLS,MT            2
GREEN BAY,WI              2
FLAGSTAFF,AZ              2
YAP- W CAROLINE IS.,PC    2
Name: count, Length: 159, dtype: int64


There is one invalid entry `"CITY"`, and every other city has exact two entries. Check the row where the `CITY` column is `CITY`:

In [193]:
print(df_sunshine_original[df_sunshine_original['CITY'] == 'CITY'])

     index  CITY  JAN  FEB  MAR  APR  MAY  JUN  JUL  AUG  SEP  OCT  NOV  DEC  \
158    158  CITY  JAN  FEB  MAR  APR  MAY  JUN  JUL  AUG  SEP  OCT  NOV  DEC   

     ANN  Unnamed: 14  
158  ANN          NaN  


In [194]:
print(df_sunshine_original.sort_values(by=['CITY','index']).head(10))

     index            CITY  JAN  FEB  MAR  APR  MAY  JUN JUL AUG SEP OCT NOV  \
115    115     ABERDEEN,SD  NaN   54   58   63   65   66  74  78  68  48  21   
315    315     ABERDEEN,SD  NaN   54   58   63   65   66  74  78  68  48  21   
123    123      ABILENE,TX   63   66   70   71   71   77  80  75  69  68  64   
182    182      ABILENE,TX   63   66   70   71   71   77  80  75  69  68  64   
86      86       ALBANY,NY   46   52   51   55   53   55  62  58  54  46  33   
287    287       ALBANY,NY   46   52   51   55   53   55  62  58  54  46  33   
84      84  ALBUQUERQUE,NM   73   73   73   78   80   82  76  76  77  80  75   
169    169  ALBUQUERQUE,NM   73   73   73   78   80   82  76  76  77  80  75   
108    108    ALLENTOWN,PA  NaN  NaN  NaN  NaN  NaN  NaN  90  93  82  52  47   
314    314    ALLENTOWN,PA  NaN  NaN  NaN  NaN  NaN  NaN  90  93  82  52  47   

     DEC  ANN  Unnamed: 14  
115  NaN  NaN          NaN  
315  NaN  NaN          NaN  
123   65   69          NaN  
182

In this data set, the row where `'index'` is 158 does not include appopriate values thus you can remvoe it.

See the `"index"` field of every pairs of rows for every city; one of those rows have the value of less than 158 and the other rows have the value of greater than 158. Also all other values such as `"ANN"` are the same.

Assume that this data set includes two tables with the same rows in different orders. The first table spans between line 1 and line 157 in the original CSV file, and the second table spans between line 158 and line 318. You only need the first one.

Construct a new DataFrame that is based on the original data set for the sunshine information with the following changes:

* Include all rows of the first 157 records.
* Trim off the unnecessary column `"Unnamed: 14"`.
* Split the "`CITY`" entries by the city names and the state abbreviations.

In [195]:
df_sunshine = df_sunshine_original\
    .iloc[1:157]\
    .set_index(['index'])

del df_sunshine['Unnamed: 14']

df_sunshine[['CITY', 'STATE']] = \
    df_sunshine['CITY'].str.split(',', expand=True)

# del df_sunshine['CITY']

print(df_sunshine.head())

             CITY JAN FEB MAR APR MAY JUN JUL AUG SEP OCT NOV DEC ANN STATE
index                                                                      
1      MONTGOMERY  47  55  58  64  63  64  61  61  59  63  55  49  58    AL
2       ANCHORAGE  43  46  51  50  51  46  43  43  41  36  35  33  43    AK
3          JUNEAU  39  35  38  42  44  37  33  35  27  21  26  21  33    AK
4            NOME  38  56  54  52  52  43  39  34  38  35  30  36  42    AK
5       FLAGSTAFF  71  73  72  82  83  88  74  75  79  77  72  76  76    AZ


In [196]:
del df_sunshine_original

### 3.2 &emsp; Data Set 2: Photovoltanic

In [197]:
df_photovoltanic_original = pd.read_csv(
    './data/uspvdb_v3_0_20250430.csv'
    )

df_photovoltanic = pd.DataFrame()

df_photovoltanic = df_photovoltanic_original[[
    'case_id', 'p_county', 'p_state', 'xlong', 'ylat', 'p_cap_ac', 'p_cap_dc'
    ]].set_index('case_id')

df_photovoltanic.rename(
    columns={
        'p_county': 'County',
        'p_state':  'State',
        'xlong':    'Longitude',
        'ylat':     'Latitude',
        'p_cap_ac': 'AC',
        'p_cap_dc': 'DC'
        },
    inplace=True
    )

df_photovoltanic['Current'] = df_photovoltanic['AC'] + df_photovoltanic['DC'] 

print(df_photovoltanic.head())

                    County State   Longitude   Latitude    AC     DC  Current
case_id                                                                      
406374   Matanuska-Susitna    AK -149.789413  61.587349   6.0    8.4     14.4
405016    Northwest Arctic    AK -162.553146  66.838470   1.7    3.4      5.1
401476          Lauderdale    AL  -87.838394  34.833809  75.0  100.2    175.2
401865                Dale    AL  -85.729469  31.331732  10.6   12.7     23.3
401866             Calhoun    AL  -85.940590  33.626301   7.4    9.7     17.1


### 3.3 &emsp; Dataset 3: Land Values

In this original data set, there are many columns of historical prices of the average home values of 3-bedroom houses but you only need the latest values: `"2025-05-31"`.

See the first few rows of the original data set while excluding all the other columns for historical home values:

In [198]:
df_landvalue_original = pd.read_csv(
    './data/City_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv.zip',
    compression='zip'
    )

df_landvalue = pd.DataFrame()

df_landvalue = df_landvalue_original[[
    'RegionID', 'SizeRank', 'State', 'RegionName', 'CountyName', '2025-05-31'
    ]].set_index('RegionID')

df_landvalue['County']\
    = df_landvalue['CountyName']\
        .str.replace(r'\s* County$', '', regex=True)\
        .str.upper()
del df_landvalue['CountyName']

df_landvalue['City'] \
    = df_landvalue['RegionName']\
        .str.upper()
del df_landvalue['RegionName']

df_landvalue['Land Value'] \
    = df_landvalue['2025-05-31']
del df_landvalue['2025-05-31']

del df_landvalue_original
print(df_landvalue.head())

          SizeRank State       County         City     Land Value
RegionID                                                         
6181             0    NY       QUEENS     NEW YORK  840048.900964
12447            1    CA  LOS ANGELES  LOS ANGELES  964249.977821
39051            2    TX       HARRIS      HOUSTON  253134.060059
17426            3    IL         COOK      CHICAGO  336756.496352
6915             4    TX        BEXAR  SAN ANTONIO  235986.092899


### 1.4 &emsp; Combining Three Data Sets into One DataFrame

| Column | Land Value  | Solar   | Sunshine | (Example 1)  | (Example 2)  |
| :----- | :---------: | :-----: | :------: | :----------- | :----------- |
| City   | &#9679;     | -       | &#9679;  | Franklin     | Franklin     |
| County | &#9679;     | &#9679; | -        | Williamson   | Williamson   |
| State  | &#9679;     | &#9679; | &#9679;  | Tennessee    | Texas        |


Note that in the United States, city names are unique only within the same state. Some cities share the same name and even the same county name while those are located in different states.